In [2]:
from imutils import face_utils
import numpy as np
import imutils
import dlib
import cv2
import random
import os
import shutil
from easydl import clear_output
# import tensorflow as tf
 

os.chdir('C:/Users/Tibame_T14/Documents/AI實作應用班/專題構想/人臉資料集/VGGface_MTCNN_process/process_image/select_images')

 
face_cascade = cv2.CascadeClassifier('./cascades/haarcascade_frontalface_default.xml')
 
# def detect_haar(filename):
#     img = cv2.imread(filename)
#     gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#     faces = face_cascade.detectMultiScale(gray,
#                                           scaleFactor=1.1,
#                                           minNeighbors=3)
#     for (x,y,w,h) in faces:
#         img = cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),3)
#     cv2.imwrite('./haar.png', img)


detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
 
def detect_face_landmarks(filename):
    img = cv2.imread(filename)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = detector(gray, 1)

    if len(faces) == 0:
        min_max_size=[0,0,0,0]
        centerx = 0
        leftx = 0
        rightx =0
    else:

 
        for face in faces:
            shape = predictor(gray, face)
            shape = face_utils.shape_to_np(shape)


            (x, y, w, h) = face_utils.rect_to_bb(face)
            cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 3)

            count=0
            min_max_size=[]
            markx=[]
            marky=[]
            centerx=0
            leftx=0
            rightx=0
            for (x, y) in shape:

                if count >= 1 and count <=15:
    #                 print(x,y)

                    markx.append(x)
                    marky.append(y)
                    cv2.circle(img, (x, y), 5, (0, 0, 255), -2)
                    if count == 4:
                        leftx = x
                    elif count == 13:
                        rightx = x
                # 圖像、圓心、半徑、顏色、第五個參數正數為線的粗細，負數則為填滿
                elif count == 51:
                    centerx= x
                    cv2.circle(img, (x, y), 5, (0, 0, 255), -2)

                count += 1

        cv2.imwrite('./face_landmarks.png', img)

        min_max_size.append(min(markx))
        min_max_size.append(max(markx))
        min_max_size.append(min(marky))
        min_max_size.append(max(marky))


    return (min_max_size[1]-min_max_size[0]), (min_max_size[3]-min_max_size[2]), min_max_size[0], min_max_size[2], centerx, leftx, rightx


os.chdir('C:/Users/Tibame_T14/Documents/AI實作應用班/專題構想/人臉資料集/VGGface_MTCNN_process/process_image/select_images')
folder_list=os.listdir()
folder_list.sort()


for folder in folder_list:
    
    file_move_destination='C:/Users/Tibame_T14/Documents/AI實作應用班/專題構想/人臉資料集/VGGface_MTCNN_process/mask_images/{}'.format(folder)
    
    if not os.path.exists(file_move_destination):
        os.mkdir(file_move_destination)
    
    
    code_work_path='C:/Users/Tibame_T14/Documents/AI實作應用班/專題構想/人臉資料集/VGGface_MTCNN_process/process_image/select_images/{}'.format(folder)
    
    os.chdir(code_work_path)
    
    print(folder)
    
    img_list=os.listdir()
    img_list.sort()
    
    process_count=0

    for img_name in img_list:

        if '.jpg' not in img_name or 'mask' in img_name:
            continue
        
        print(folder,img_name)
        

            
        process_count += 1

        filename = img_name
        # detect_haar(filename)
        xsize, ysize, xstart, ystart, centerx, leftx, rightx = detect_face_landmarks(filename)

        if xsize == 0:
            os.remove(filename)
            continue


        left_dist=(centerx-leftx)
        right_dist=abs(centerx-rightx)
        img_red=0
        img_st=0

        #白口罩
        high_Lfacemask=['LFM3.png','LFM4.png'] 
        high_Cfacemask=['CFM1.png','CFM5.png']
        #立體口罩
        d3_Lfacemask = ['LFM8.png','LFM9.png','LFM10.png','LFM11.png','LFM12.png','LFM13.png'] 

        if left_dist+70 < right_dist:
            os.chdir('C:/Users/Tibame_T14/Documents/AI實作應用班/專題構想/model/left_facemask_data')
            mask_list = os.listdir()
            mask_name = random.choice(mask_list)
    #         print('left')
            img_red = 30
            img_st = 15
            back_threshold=250
            if mask_name in high_Lfacemask:
                back_threshold=254

            fmask = cv2.imread(mask_name)
            fmask = cv2.resize(fmask,(xsize - img_red,ysize))

        elif left_dist > right_dist + 70:
            os.chdir('C:/Users/Tibame_T14/Documents/AI實作應用班/專題構想/model/right_facemask_data')
            mask_list = os.listdir()
            mask_name = random.choice(mask_list)
    #         print('right')
            img_red = 0
            img_st = -25
            back_threshold=250
            if mask_name == 'RFM2.png':
                back_threshold=254
            fmask = cv2.imread(mask_name)
            fmask = cv2.resize(fmask,(xsize - img_red,ysize))
        else:
            os.chdir('C:/Users/Tibame_T14/Documents/AI實作應用班/專題構想/model/center_facemask_data')
            mask_list = os.listdir()
            mask_name = random.choice(mask_list)
    #         print('center')
            img_red = 0
            img_st = -5
            back_threshold=250
            if mask_name in high_Cfacemask:
                back_threshold=254
            fmask = cv2.imread(mask_name)
            fmask = cv2.resize(fmask,(xsize - img_red,ysize))

        os.chdir('C:/Users/Tibame_T14/Documents/AI實作應用班/專題構想/人臉資料集/VGGface_MTCNN_process/process_image/select_images/{}'.format(folder))
        man = cv2.imread(filename)

        #2. 將要P的圖，使用漫水填充法，濾出背景 (cv2.floodFill)
        copyfmask = fmask.copy()
        h, w = fmask.shape[:2]
        mask = np.zeros([h+2, w+2], np.uint8)
        try:
            cv2.floodFill(copyfmask, mask, (30, 30), (255, 255, 255), (100, 100, 100), (50, 50, 50), cv2.FLOODFILL_FIXED_RANGE)  
        except:
            os.remove(filename)
            continue

        #3. 將濾出背景的圖，挖出人物，取得遮罩(黑色)

        fmaskgray = cv2.cvtColor(fmask,cv2.COLOR_BGR2GRAY)
        ret, mask = cv2.threshold(fmaskgray, back_threshold, 255, cv2.THRESH_BINARY)


        # 4. 將遮罩印在原來的圖片上，有點像挖空的感覺
        if (xsize+xstart) > man.shape[0]:
            xstart = xstart - ((xsize+xstart)-man.shape[0])
        if (ysize+ystart) > man.shape[0]:
            ystart = ystart - ((ysize+ystart)-man.shape[0])


        rows, cols, channels = fmask.shape
        x = xstart - img_st
        y = ystart
        roi = man[y:y+rows, x:x+cols]

        # print(y,y+rows)

        # print("Use mask to crop origin image1:")
        try:
            man_bg = cv2.bitwise_and(roi, roi, mask = mask)
        except:
            os.remove(filename)
            continue

        #5. 挖空後，反向操作取得原先要P圖的人物

        mask_inv = cv2.bitwise_not(mask)
        fmask_fg = cv2.bitwise_and(fmask, fmask, mask = mask_inv)


        #6. 合併至原來的圖片 

        dst = cv2.add(man_bg,fmask_fg)
        man[y:y+rows, x:x+cols] = dst

        save_name='./'+img_name.split('.')[0]+'_mask.jpg'
        cv2.imwrite(save_name, man)

        shutil.move(img_name,file_move_destination)
        shutil.move(save_name,file_move_destination)

        if process_count % 20 == 0:
            clear_output()
     
    rest_img=os.listdir()
    if len(rest_img) == 1 and 'face_landmarks.png' in rest_img:
        shutil.move('face_landmarks.png', file_move_destination)
        os.chdir('../')
        os.rmdir(code_work_path)
        
    clear_output()


ModuleNotFoundError: No module named 'imutils'

In [31]:
from imutils import face_utils
import numpy as np
import imutils
import dlib
import cv2
import random
import os
import shutil
from easydl import clear_output


os.chdir('C:/Users/Tibame_T14/Documents/AI實作應用班/專題構想/人臉資料集/VGGface_MTCNN_process/process_image/select_images')

 
face_cascade = cv2.CascadeClassifier('./cascades/haarcascade_frontalface_default.xml')

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")




os.chdir('C:/Users/Tibame_T14/Documents/AI實作應用班/專題構想/人臉資料集/VGGface_MTCNN_process/process_image/select_images/n000438')

filename = './0420_01.jpg'
  
img = cv2.imread(filename)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
faces = detector(gray, 1)

print(len(faces))

if len(faces) == 0:
    min_max_size=[0,0,0,0]
else:

    for face in faces:
#         print('hi')

        shape = predictor(gray, face)
        shape = face_utils.shape_to_np(shape)


        (x, y, w, h) = face_utils.rect_to_bb(face)
        cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 3)

        count=0
        min_max_size=[]
        markx=[]
        marky=[]
        centerx=0
        leftx=0
        rightx=0
    #     print(count)
        for (x, y) in shape:


            if count >= 1 and count <=15:
#                 print(x,y)

                markx.append(x)
                marky.append(y)
                cv2.circle(img, (x, y), 5, (0, 0, 255), -2)
                if count == 4:
                    leftx = x
                elif count == 13:
                    rightx = x
            # 圖像、圓心、半徑、顏色、第五個參數正數為線的粗細，負數則為填滿
            elif count == 51:
                centerx= x
                cv2.circle(img, (x, y), 5, (0, 0, 255), -2)

            count += 1

#     print(len(markx),len(marky))
    cv2.imwrite('./face_landmarks.png', img)

    if markx and marky:
        min_max_size.append(min(markx))
        min_max_size.append(max(markx))
        min_max_size.append(min(marky))
        min_max_size.append(max(marky))
# #     else:
# #         min_max_size=[0,0,0,0]

#     print(markx, marky)

print(min_max_size[1]-min_max_size[0], (min_max_size[3]-min_max_size[2]), min_max_size[0], \
min_max_size[2], centerx, leftx, rightx)
print(min_max_size)

0
0 0 0 0 271 85 378
[0, 0, 0, 0]


In [ ]:
print(len(markx))